# 传统机器学习方法
路线：前置准备 预处理 可视化 特征提取 特征选择 分类器

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch, lfilter
from copy import deepcopy
from entropy import *
import entropy
from load_lyh_data import  *

## 0. 前置准备
加载lyh的数据，这里仅仅是加载数据对数据进行分割，并未对数据进行任何处理

In [2]:
raw_data_dict = read_folder_npy_data()

Loaded nothing1_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing2_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing3_orginal.npy: Shape=(100,), Dtype=object
Loaded left1_orginal.npy: Shape=(100,), Dtype=object
Loaded left2_orginal.npy: Shape=(100,), Dtype=object
Loaded left3_orginal.npy: Shape=(100,), Dtype=object
Loaded right1_orginal.npy: Shape=(100,), Dtype=object
Loaded right2_orginal.npy: Shape=(100,), Dtype=object
Loaded right3_orginal.npy: Shape=(100,), Dtype=object
Loaded leg1_orginal.npy: Shape=(100,), Dtype=object
Loaded leg2_orginal.npy: Shape=(100,), Dtype=object
Loaded leg3_orginal.npy: Shape=(100,), Dtype=object


In [5]:
# 参数设计

# 下采样设置：ERP脑电有效频率<30Hz，根据脑奎斯特定理，
# 采样率到60Hz以上即可。但为了多一些保存细节，这里降采样设置为125Hz
fs_raw = 250
fs_down = 125
# 一个trial的时间
t_of_trial = 4
# 采样点个数
nTime = fs_down * t_of_trial

start_point_index = 2
end_point_index = start_point_index + fs_raw * t_of_trial


def down_sample(data, start, end, fs_raw, fs_down):
    '''
    下采样
    param---
    data: numpy数组
    return---
    data: 下采样之后的数组
    '''
    
    step = fs_raw // fs_down
    data = data[start: end: step]
    
    return data

In [7]:
# 选择channel
channel_index_list = np.arange(2, 2+14)

def selected_channel(data, channel_index_list):

    return data[:, channel_index_list]

In [16]:
file_label_dict = {
    "nothing1_orginal.npy": 0, "nothing2_orginal.npy": 0, "nothing3_orginal.npy": 0,
    "left1_orginal.npy": 1, "left2_orginal.npy": 1, "left3_orginal.npy": 1,
    "right1_orginal.npy": 2, "right2_orginal.npy": 2, "right3_orginal.npy": 2,
    "leg1_orginal.npy": 3, "leg2_orginal.npy": 3, "leg3_orginal.npy": 3
}
save_path = os.getcwd() + "/lyh_data/session1/Standard_input"
if not os.path.exists(save_path):
    os.makedirs(save_path)

def pre_preparation(data_dict, file_label_dict, save_path, nClasses=4):
    data = [[] for _ in range(nClasses)]

    for key, value in data_dict.items():
        d = data[file_label_dict[key]]
        for trial in value:
            trial = down_sample(np.array(trial), start_point_index, end_point_index, fs_raw, fs_down)
            trail = selected_channel(trial, channel_index_list)
            d.append(trail.T)
    
    # save
    for i, d in enumerate(data):
        data[i] = np.array(d)
        path = save_path + "/" + str(i) + ".npy"
        np.save(path, data[i])

    return data

In [73]:
standard_input_data_list = pre_preparation(raw_data_dict, file_label_dict, save_path)

In [18]:
standard_input_data_list[0].shape

(300, 14, 500)

## 1. 预处理

In [74]:
# 基线校正
def baseline_correction(data, baseline_start, baseline_end):
    """
    进行基线校正

    Parameters:
    - data: EEG 数据的二维数组，shape=(通道数, 采样点数)
    - baseline_start: 基线起始点的索引
    - baseline_end: 基线结束点的索引

    Returns:
    - baseline_corrected_data: 基线校正后的 EEG 数据
    """
    # 计算每个通道上基线期的平均值
    baseline_values = np.mean(data[:, baseline_start:baseline_end], axis=1, keepdims=True)

    # 进行基线校正
    baseline_corrected_data = data - baseline_values

    return baseline_corrected_data

In [75]:
# 作为基线的比例
baseline_ratio = 0.1
baseline_start = 0
baseline_end = int(0.1 * nTime)

x = deepcopy(standard_input_data_list[0][0])
print("基线校正前 : ", x)
x = baseline_correction(x, baseline_start, baseline_end)
print("基线校正后 : ", x)

基线校正前 :  [[4298.333 4301.538 4307.564 ... 4320.    4332.436 4314.872]
 [4268.462 4269.103 4268.462 ... 4282.821 4301.41  4287.051]
 [4328.462 4324.103 4328.333 ... 4351.795 4355.897 4346.667]
 ...
 [4463.077 4457.821 4469.615 ... 4502.308 4510.128 4505.897]
 [4335.641 4342.179 4357.308 ... 4426.795 4437.692 4419.231]
 [4418.846 4415.769 4421.154 ... 4469.615 4474.231 4469.744]]
基线校正后 :  [[ 11.42274  14.62774  20.65374 ...  33.08974  45.52574  27.96174]
 [ 23.91832  24.55932  23.91832 ...  38.27732  56.86632  42.50732]
 [ 14.37734  10.01834  14.24834 ...  37.71034  41.81234  32.58234]
 ...
 [  0.56158  -4.69442   7.09958 ...  39.79258  47.61258  43.38158]
 [-26.3051  -19.7671   -4.6381  ...  64.8489   75.7459   57.2849 ]
 [-10.63602 -13.71302  -8.32802 ...  40.13298  44.74898  40.26198]]


In [76]:
standard_input_data_list[0][0]

array([[4298.333, 4301.538, 4307.564, ..., 4320.   , 4332.436, 4314.872],
       [4268.462, 4269.103, 4268.462, ..., 4282.821, 4301.41 , 4287.051],
       [4328.462, 4324.103, 4328.333, ..., 4351.795, 4355.897, 4346.667],
       ...,
       [4463.077, 4457.821, 4469.615, ..., 4502.308, 4510.128, 4505.897],
       [4335.641, 4342.179, 4357.308, ..., 4426.795, 4437.692, 4419.231],
       [4418.846, 4415.769, 4421.154, ..., 4469.615, 4474.231, 4469.744]])

In [10]:
# 带通滤波器 0.05Hz-40Hz
lowcut = 0.05
highcut = 40
fs = fs_down
order = 2

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    fa = 0.5 * fs
    low = lowcut / fa
    high = highcut / fa
    b, a = butter(order, [low, high], btype='band')
    ret = []
    for line in data:
        ret.append(filtfilt(b, a, line))
    return np.array(ret)

def iirnotch_filter(data, fs = 125, Q = 30, f_cut = 50.0):
    ret = []
    b, a = iirnotch(f_cut, Q, fs)
    for line in data:
        ret.append(lfilter(b,a, line))
    return np.array(ret)


In [66]:
# shape = (14, 500)  # 示例形状，这里是3行2列
# test1 = np.random.rand(*shape)  # 解包shape为独立的参数
# test2 = deepcopy(test1)
# print("raw data: ", test1)
# test1 = butter_bandpass_filter(test1, lowcut, highcut, fs, order)
# test1 = iirnotch_filter(test1)
# print("滤波: ", test1)
# test2 = baseline_correction(test2, baseline_start, baseline_end)
# print("基线校正: ", test2)
# test2 = butter_bandpass_filter(test2, lowcut, highcut, fs, order)
# test2 = iirnotch_filter(test2)
# print("基线校正+滤波: ", test2)

raw data:  [[0.71975731 0.76830397 0.33347613 ... 0.96665006 0.98089686 0.64238853]
 [0.45019288 0.52688822 0.97496468 ... 0.50036375 0.93432375 0.69061286]
 [0.20299125 0.19351531 0.47885269 ... 0.82760863 0.10179565 0.3866357 ]
 ...
 [0.91063828 0.81434754 0.39893596 ... 0.46398696 0.54912456 0.45694705]
 [0.93614434 0.31974904 0.90733623 ... 0.69821682 0.51476252 0.34842152]
 [0.31800987 0.81562617 0.25448075 ... 0.86655682 0.70891056 0.33324974]]
滤波:  [[-0.02321653 -0.01432984 -0.40747343 ...  0.03989705  0.27863996
  -0.09407378]
 [ 0.0481928   0.21649998  0.43359855 ... -0.63016849 -0.15558356
  -0.3685327 ]
 [ 0.06613227  0.14469515  0.20909337 ...  0.31116587  0.12106863
   0.13481327]
 ...
 [ 0.08135337 -0.01291482 -0.42700086 ...  0.00525067  0.21542023
   0.13344994]
 [-0.04829846 -0.42519548 -0.31395974 ...  0.63668495  0.39321582
   0.22054132]
 [-0.16162607  0.17503425 -0.10593489 ...  0.86422312  0.73232321
   0.37139722]]
基线校正:  [[ 0.18717069  0.23571736 -0.19911049 ...

In [72]:
# shape = (14, 500)  # 示例形状，这里是3行2列
# test1 = np.random.rand(*shape)  # 解包shape为独立的参数
# test2 = deepcopy(test1)
# shape1 = (14, 1)
# sub = np.ones(shape1)
# print(sub)
# print("raw data: ", test1)
# test1 = butter_bandpass_filter(test1, lowcut, highcut, fs, order)
# test1 = iirnotch_filter(test1)
# print("滤波: ", test1)
# test2 = test2 - sub
# print("基线校正: ", test2)
# test2 = butter_bandpass_filter(test2, lowcut, highcut, fs, order)
# test2 = iirnotch_filter(test2)
# print("基线校正+滤波: ", test2)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
raw data:  [[0.01987379 0.34003506 0.05323721 ... 0.16883784 0.30067503 0.58919963]
 [0.31803727 0.05371726 0.65359817 ... 0.86014722 0.42232539 0.48166455]
 [0.26201529 0.80920886 0.10913765 ... 0.59461616 0.89124159 0.66696895]
 ...
 [0.82313029 0.19186278 0.83968552 ... 0.23007594 0.76695717 0.48286783]
 [0.72648586 0.84872972 0.0192662  ... 0.65085979 0.48650704 0.49081444]
 [0.11020996 0.1295229  0.50470722 ... 0.81128504 0.33153466 0.00380652]]
滤波:  [[ 0.03289372  0.27890839  0.16071841 ... -0.28102633 -0.03514843
   0.20150956]
 [ 0.18461827  0.10663454  0.25867693 ...  0.44846537  0.40118385
   0.31161746]
 [-0.18206549  0.13941995 -0.12796279 ... -0.17973037  0.14465595
  -0.05966101]
 ...
 [ 0.09933024 -0.28662583 -0.16634554 ... -0.36949289 -0.02220749
  -0.17792545]
 [ 0.2084953   0.14098994 -0.20768738 ...  0.03845517 -0.13559455
  -0.15714414]
 [ 0.00142415  0.09497048  0.22805068 ...  0.

In [78]:
print("滤波前 : ", x)
x = butter_bandpass_filter(x, lowcut, highcut, fs, order)
x = iirnotch_filter(x)
print("基线矫正+滤波后 : ", x)

滤波前 :  [[ 11.42274  14.62774  20.65374 ...  33.08974  45.52574  27.96174]
 [ 23.91832  24.55932  23.91832 ...  38.27732  56.86632  42.50732]
 [ 14.37734  10.01834  14.24834 ...  37.71034  41.81234  32.58234]
 ...
 [  0.56158  -4.69442   7.09958 ...  39.79258  47.61258  43.38158]
 [-26.3051  -19.7671   -4.6381  ...  64.8489   75.7459   57.2849 ]
 [-10.63602 -13.71302  -8.32802 ...  40.13298  44.74898  40.26198]]
基线矫正+滤波后 :  [[  1.2233674    5.31136275   9.38478621 ...  23.26321155  28.70143919
   16.27871029]
 [  4.62252065   4.89188273   5.60975696 ...   7.82624006  23.85918253
   12.7586109 ]
 [ -1.11384209  -4.53790032  -1.92974007 ...  11.24382281  13.61377979
    5.92651266]
 ...
 [  0.54598765  -3.07385892   6.49437274 ...  -7.09438025  -0.34693384
   -3.45731556]
 [-11.19020058  -4.24634911   8.3426814  ...  37.09434085  42.91027207
   28.24304665]
 [ -4.26287477  -6.92866165  -2.46119829 ...   3.37859591   5.17940678
    2.14527731]]


In [79]:
print("滤波前 : ", standard_input_data_list[0][0])
y = butter_bandpass_filter(standard_input_data_list[0][0], lowcut, highcut, fs, order)
y = iirnotch_filter(y)
print("滤波后 : ", y)

滤波前 :  [[4298.333 4301.538 4307.564 ... 4320.    4332.436 4314.872]
 [4268.462 4269.103 4268.462 ... 4282.821 4301.41  4287.051]
 [4328.462 4324.103 4328.333 ... 4351.795 4355.897 4346.667]
 ...
 [4463.077 4457.821 4469.615 ... 4502.308 4510.128 4505.897]
 [4335.641 4342.179 4357.308 ... 4426.795 4437.692 4419.231]
 [4418.846 4415.769 4421.154 ... 4469.615 4474.231 4469.744]]
滤波后 :  [[  1.2233674    5.31136275   9.38478621 ...  23.26321155  28.70143919
   16.27871029]
 [  4.62252064   4.89188273   5.60975696 ...   7.82624006  23.85918253
   12.7586109 ]
 [ -1.11384209  -4.53790032  -1.92974007 ...  11.24382281  13.61377978
    5.92651266]
 ...
 [  0.54598765  -3.07385893   6.49437274 ...  -7.09438025  -0.34693384
   -3.45731556]
 [-11.19020058  -4.24634911   8.3426814  ...  37.09434085  42.91027207
   28.24304665]
 [ -4.26287477  -6.92866165  -2.46119829 ...   3.37859591   5.17940678
    2.14527731]]


In [80]:
save_path = os.getcwd() + "/lyh_data/session1/Preprocessed_data"
if not os.path.exists(save_path):
    os.makedirs(save_path)

def pre_processing(data, save_path, butter_order = 2):

    for data_per_class in data:
        for i, d in enumerate(data_per_class):
            assert d.shape == (14, 500)
            # data_per_class[i] = baseline_correction(data_per_class[i], baseline_start, baseline_end)
            data_per_class[i] = butter_bandpass_filter(data_per_class[i], lowcut, highcut, fs, order)
            data_per_class[i] = iirnotch_filter(data_per_class[i])
    
    # save
    for i, d in enumerate(data):
        data[i] = np.array(d)
        path = save_path + "/" + str(i) + ".npy"
        np.save(path, data[i])

In [81]:
print("滤波前：", standard_input_data_list[0][0])
pre_processing(standard_input_data_list, save_path)
print("滤波后: ", standard_input_data_list[0][0])

滤波前： [[4298.333 4301.538 4307.564 ... 4320.    4332.436 4314.872]
 [4268.462 4269.103 4268.462 ... 4282.821 4301.41  4287.051]
 [4328.462 4324.103 4328.333 ... 4351.795 4355.897 4346.667]
 ...
 [4463.077 4457.821 4469.615 ... 4502.308 4510.128 4505.897]
 [4335.641 4342.179 4357.308 ... 4426.795 4437.692 4419.231]
 [4418.846 4415.769 4421.154 ... 4469.615 4474.231 4469.744]]
滤波后:  [[  1.2233674    5.31136275   9.38478621 ...  23.26321155  28.70143919
   16.27871029]
 [  4.62252064   4.89188273   5.60975696 ...   7.82624006  23.85918253
   12.7586109 ]
 [ -1.11384209  -4.53790032  -1.92974007 ...  11.24382281  13.61377978
    5.92651266]
 ...
 [  0.54598765  -3.07385893   6.49437274 ...  -7.09438025  -0.34693384
   -3.45731556]
 [-11.19020058  -4.24634911   8.3426814  ...  37.09434085  42.91027207
   28.24304665]
 [ -4.26287477  -6.92866165  -2.46119829 ...   3.37859591   5.17940678
    2.14527731]]


## 2. 特征提取


### 2.1 时域特征提取

In [16]:
len(standard_input_data_list), len(standard_input_data_list[0])

NameError: name 'standard_input_data_list' is not defined

In [17]:
dataset_np = np.array(standard_input_data_list)
dataset_np = dataset_np.reshape(-1, 14, 500)
dataset_np.shape

NameError: name 'standard_input_data_list' is not defined

In [84]:
dataset_list = list(dataset_np)
label_list = [0] * 300 + [1] * 300 + [2] * 300 + [3] * 300
data_df = pd.DataFrame({'raw data': dataset_list, 'label': label_list})
x = deepcopy(data_df.iloc[0])

In [14]:
test_df = pd.DataFrame({'raw data': [test], 'label': [0]})

In [18]:
# 计算过零率

def zero_crossing_rate(trials):
    '''
    计算一个trials(二维列表)各个通道的过零率, 返回一个list
    '''

    def compute(signal):
        '''
        计算一维信号的过零率
        '''
        crossings = np.where(np.diff(np.sign(signal)))[0]
        zero_crossing_rate = len(crossings) / len(signal)
        return zero_crossing_rate

    ret = [compute(trials[i]) for i in range(len(trials))]
    return np.array(ret)

In [86]:
# 同一类别的过零率差别大
np.array(zero_crossing_rate(x['raw data'])),\
     np.array(zero_crossing_rate(data_df.iloc[1]['raw data'])),\
       np.array(zero_crossing_rate(data_df.iloc[2]['raw data'])  )

(array([0.168, 0.04 , 0.13 , 0.102, 0.198, 0.15 , 0.14 , 0.186, 0.126,
        0.074, 0.106, 0.114, 0.062, 0.046]),
 array([0.15 , 0.088, 0.156, 0.084, 0.108, 0.112, 0.174, 0.184, 0.18 ,
        0.136, 0.108, 0.184, 0.08 , 0.104]),
 array([0.138, 0.1  , 0.078, 0.07 , 0.05 , 0.154, 0.184, 0.09 , 0.172,
        0.094, 0.12 , 0.124, 0.062, 0.112]))

In [87]:
# 同一类别的过零率差别大
np.array(zero_crossing_rate(data_df.iloc[398]['raw data'])),\
     np.array(zero_crossing_rate(data_df.iloc[399]['raw data'])),\
       np.array(zero_crossing_rate(data_df.iloc[400]['raw data'])  )

(array([0.056, 0.084, 0.112, 0.106, 0.05 , 0.066, 0.062, 0.17 , 0.136,
        0.076, 0.128, 0.138, 0.1  , 0.084]),
 array([0.106, 0.056, 0.098, 0.054, 0.028, 0.04 , 0.06 , 0.104, 0.076,
        0.064, 0.032, 0.068, 0.046, 0.076]),
 array([0.012, 0.008, 0.016, 0.054, 0.026, 0.014, 0.018, 0.058, 0.05 ,
        0.042, 0.054, 0.032, 0.034, 0.048]))

In [19]:
# 计算均值

def calculate_channel_means(eeg_data):
    """
    计算每个通道的均值。

    参数：
    - eeg_data: 二维 NumPy 数组，表示 EEG 数据，形状为 (n_channels, n_points)。

    返回：
    - channel_means: 一维 NumPy 数组，包含每个通道的均值。
    """
    # 计算每个通道的均值，axis=2 表示在样本点上求均值
    channel_means = np.mean(eeg_data, axis=1)

    return channel_means


In [89]:
# 同一类别的trials均值差别怎么这么大？
calculate_channel_means(data_df.iloc[0]['raw data']), calculate_channel_means(data_df.iloc[1]['raw data'])

(array([  5.79579612,  -6.37368547,  -1.11140977, -10.55618282,
          2.01619029,  -1.79062357,  -1.40401936,   0.80076936,
          3.03901338,   4.1266799 ,  11.40424161,  -0.07762688,
         26.9461035 ,  -1.1365759 ]),
 array([ -7.05026265, -18.51313891,  -5.52623322,  -6.18110454,
          5.75653787,  10.04083053,  -0.65157368,   0.92565608,
         -1.22593159,  -4.27781215,   6.5364136 ,   3.26128473,
          6.60940732,   5.42792899]))

In [20]:
# 计算标准差

def calculate_channel_stds(eeg_data):
    """
    计算每个通道的均值。

    参数：
    - eeg_data: 二维 NumPy 数组，表示 EEG 数据，形状为 (n_channels, n_points)。

    返回：
    - channel_means: 一维 NumPy 数组，包含每个通道的均值。
    """
    # 计算每个通道的均值，axis=2 表示在样本点上求均值
    channel_stds = np.std(eeg_data, axis=1)

    return channel_stds

In [21]:
def calculate_first_order_diff(eeg_data):

    first_order_diff = np.sum(np.abs(np.diff(eeg_data, axis=1)), axis=1) / (eeg_data.shape[1] - 1)

    return first_order_diff

def calculate_second_order_diff(eeg_data):
    
    second_order_diff = np.sum(np.abs(np.diff(eeg_data, n=2, axis=1)), axis=1) / (eeg_data.shape[1] - 2)

    return second_order_diff


In [22]:
def time_domain_feature(eeg_dataset):
    '''
    提取时域特征
    params:
    - eeg_dataset: Dataframe类型
    '''
    zero_crossing_list = []
    means_list = []
    stds_list = []
    first_order_diff_list = []
    second_order_diff_list = []

    for _, row in eeg_dataset.iterrows():
        zero_crossing = zero_crossing_rate(row['raw data'])
        zero_crossing_list.append(zero_crossing)
        means = calculate_channel_means(row['raw data'])
        means_list.append(means)
        stds = calculate_channel_stds(row['raw data'])
        stds_list.append(stds)
        first_diff = calculate_first_order_diff(row['raw data'])
        first_order_diff_list.append(first_diff)
        second_diff = calculate_second_order_diff(row['raw data'])
        second_order_diff_list.append(second_diff)
    
    nChannels = zero_crossing_list[0].shape[0]
    for i in range(nChannels):
        eeg_dataset[f'zero_crossing_rate_c{i+1}'] = np.array(zero_crossing_list)[:,i]
        eeg_dataset[f'mean_c{i+1}'] = np.array(means_list)[:,i]
        eeg_dataset[f'std_c{i+1}'] = np.array(stds_list)[:,i]
        eeg_dataset[f'first_order_diff_c{i+1}'] = np.array(first_order_diff_list)[:,i]
        eeg_dataset[f'second_order_diff_c{i+1}'] = np.array(second_order_diff_list)[:,i]
        

In [93]:
time_domain_feature(data_df)

In [94]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 72 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   raw data                1200 non-null   object 
 1   label                   1200 non-null   int64  
 2   zero_crossing_rate_c1   1200 non-null   float64
 3   mean_c1                 1200 non-null   float64
 4   std_c1                  1200 non-null   float64
 5   first_order_diff_c1     1200 non-null   float64
 6   second_order_diff_c1    1200 non-null   float64
 7   zero_crossing_rate_c2   1200 non-null   float64
 8   mean_c2                 1200 non-null   float64
 9   std_c2                  1200 non-null   float64
 10  first_order_diff_c2     1200 non-null   float64
 11  second_order_diff_c2    1200 non-null   float64
 12  zero_crossing_rate_c3   1200 non-null   float64
 13  mean_c3                 1200 non-null   float64
 14  std_c3                  1200 non-null   

### 2.2 频域特征提取

In [23]:
from scipy.signal import welch

def five_band_energy(eeg_data, fs=125):
    
    energy = []
    
    for eeg_signal in eeg_data:
        # 计算功率谱密度（PSD）
        frequencies, psd = welch(eeg_signal, fs, nperseg=1024)

        # 定义频带边界
        delta_band = (0.5, 4)
        theta_band = (4, 8)
        alpha_band = (8, 14)
        beta_band = (14, 30)
        gamma_band = (30, 60)

        # 计算每个频带内的能量
        energy.append(np.trapz(psd[(frequencies >= delta_band[0]) & (frequencies <= delta_band[1])], \
            frequencies[(frequencies >= delta_band[0]) & (frequencies <= delta_band[1])]))
        energy.append(np.trapz(psd[(frequencies >= theta_band[0]) & (frequencies <= theta_band[1])], \
            frequencies[(frequencies >= theta_band[0]) & (frequencies <= theta_band[1])]))
        energy.append(np.trapz(psd[(frequencies >= alpha_band[0]) & (frequencies <= alpha_band[1])], \
            frequencies[(frequencies >= alpha_band[0]) & (frequencies <= alpha_band[1])]))
        energy.append(np.trapz(psd[(frequencies >= beta_band[0]) & (frequencies <= beta_band[1])], \
            frequencies[(frequencies >= beta_band[0]) & (frequencies <= beta_band[1])]))
        energy.append(np.trapz(psd[(frequencies >= gamma_band[0]) & (frequencies <= gamma_band[1])], \
            frequencies[(frequencies >= gamma_band[0]) & (frequencies <= gamma_band[1])]))

    return np.array(energy)


In [24]:
def freq_domain_feature(eeg_dataset):
    '''
    提取时域特征

    参数:
    - eeg_dataset: Dataframe类型
    '''
    
    energy = []

    for _, row in eeg_dataset.iterrows():
        
        energy.append(five_band_energy(row['raw data']))

    freq_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    nFreqs = len(freq_names)
    nChannels = energy[0].shape[0] // nFreqs
    
    for i in range(nChannels):
        for j in range(nFreqs):
            eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]

In [97]:
freq_domain_feature(data_df)

/home/pai/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 500, using nperseg = 500
  warnings.warn('nperseg = {0:d} is greater than input length '
/tmp/ipykernel_390/1711663099.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/tmp/ipykernel_390/1711663099.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/

In [25]:
time_domain_feature(test_df)
freq_domain_feature(test_df)

/home/pai/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 500, using nperseg = 500
  warnings.warn('nperseg = {0:d} is greater than input length '
/tmp/ipykernel_26126/1711663099.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/tmp/ipykernel_26126/1711663099.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+

,raw data,label,zero_crossing_rate_c1,mean_c1,std_c1,first_order_diff_c1,second_order_diff_c1,zero_crossing_rate_c2,mean_c2,std_c2,...,delta_c13,theta_c13,alpha_c13,beta_c13,gamma_c13,delta_c14,theta_c14,alpha_c14,beta_c14,gamma_c14
0,"[[1.2233674002786974, 5.311362745323572, 9.384...",0,0.168,5.795796,26.747689,6.917654,7.757805,0.04,-6.373685,40.206396,...,232.707939,30.929507,19.584694,27.794255,10.281614,380.16133,25.192386,8.704919,11.973297,4.960753


In [26]:
test_df['raw data']

0    [[1.2233674002786974, 5.311362745323572, 9.384...
Name: raw data, dtype: object

In [98]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Columns: 142 entries, raw data to gamma_c14
dtypes: float64(140), int64(1), object(1)
memory usage: 1.3+ MB


In [99]:
save_path = os.getcwd() + "/lyh_data/session1/Feature_extracted"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
save_path += "/lyh_feature_extracted.csv"
    
data_df.to_csv(save_path, index=False)

## 3. 分类模型

In [144]:
# 构造数据集
from sklearn.model_selection import train_test_split

# 打乱数据集
shuffled_df = data_df.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

In [109]:
train_id = train_df.index.to_list()
test_id = test_df.index.to_list()
for item in train_id:
    if item in test_id:
        print("存在相等的元素")
        break
else:
    print("不存在相等的元素")

不存在相等的元素


In [110]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((960, 140), (960,), (240, 140), (240,))

In [111]:
from lightgbm import LGBMClassifier

lgm = LGBMClassifier()
lgm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33576
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 140
[LightGBM] [Info] Start training from score -1.382136
[LightGBM] [Info] Start training from score -1.365675
[LightGBM] [Info] Start training from score -1.361602
[LightGBM] [Info] Start training from score -1.437588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

LGBMClassifier()

In [112]:
# 手工特征提取+未调参LightGBM
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.9125)

In [113]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)
print("confusion_matrix:\n", cm)

confusion_matrix:
 [[56  1  0  2]
 [ 6 46  1  2]
 [ 0  0 54  0]
 [ 6  2  1 63]]


In [117]:
# 原始数据进行训练
shuffled_df = data_df.sample(frac=1, random_state=42)
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)

X_train = np.array(train_df['raw data'])
print(X_train[0])
X_test = np.array(test_df['raw data'])
X_train = np.vstack(X_train).reshape(-1, 14, 500)
X_test = np.vstack(X_test).reshape(-1, 14, 500)
X_train = X_train.reshape(-1, 14 * 500)
X_test = X_test.reshape(-1, 14 * 500)
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])
X_train[0]


[[  5.1359676   -3.56294757  -4.93683818 ... -24.83999672 -20.72275314
  -18.75059175]
 [ -1.57088852 -10.06453023 -10.46421132 ... -38.55049664 -27.24653005
  -23.78624649]
 [  0.55504042  -7.72564738  -4.94103614 ... -20.07059185 -20.46490629
  -13.69915833]
 ...
 [ -4.28028626 -10.41083709  -7.68718556 ... -13.32182841 -16.75106415
  -10.86976717]
 [  6.45557612   9.23443594  15.71743304 ... -55.61471825 -51.56785603
  -36.44763422]
 [ -2.82070615  -1.63619278   1.08530902 ...  -1.0391331   -4.03671668
   -0.33247381]]


array([ 5.1359676 , -3.56294757, -4.93683818, ..., -1.0391331 ,
       -4.03671668, -0.33247381])

In [118]:
lgm = LGBMClassifier()
lgm.fit(X_train, y_train)

# 原始特征+未调参LightGBM
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.270247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785000
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 7000
[LightGBM] [Info] Start training from score -1.382136
[LightGBM] [Info] Start training from score -1.365675
[LightGBM] [Info] Start training from score -1.361602
[LightGBM] [Info] Start training from score -1.437588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


(1.0, 0.4875)

In [42]:
from xgboost import XGBClassifier
# 原始数据进行训练

shuffled_df = data_df.sample(frac=1, random_state=42)
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)

X_train = np.array(train_df['raw data'])
X_test = np.array(test_df['raw data'])
X_train = np.vstack(X_train).reshape(-1, 14, 500)
X_test = np.vstack(X_test).reshape(-1, 14, 500)
X_train = X_train.reshape(-1, 14 * 500)
X_test = X_test.reshape(-1, 14 * 500)
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

xgb = XGBClassifier()
xgb.fit(X_train, y_train)


# 原始特征+未调参xgboost
y_train_pred = xgb.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = xgb.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.48333333333333334)

In [131]:
# 构造数据集
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 打乱数据集
shuffled_df = data_df.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
# xgb.save_model("xgb.json")

# 手工特征提取+未调参XGBoost
y_train_pred = xgb.predict(X_train)
# acc_train = (y_train == y_train_pred).sum() / len(y_train)
acc_train = accuracy_score(y_train, y_train_pred)

y_test_pred = xgb.predict(X_test)
# acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_test = accuracy_score(y_test, y_test_pred)

acc_train, acc_test

(1.0, 0.925)

In [139]:
clf = XGBClassifier()
clf.load_model('./xgb_4s_se1.json')

In [143]:
clf.predict([np.ones(shape=(140,))])

array([2])

In [133]:
y_train_pred = clf.predict(X_train)
# acc_train = (y_train == y_train_pred).sum() / len(y_train)
acc_train = accuracy_score(y_train, y_train_pred)

y_test_pred = clf.predict(X_test)
# acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_test = accuracy_score(y_test, y_test_pred)

acc_train, acc_test

(1.0, 0.925)

In [142]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2560 entries, 1172 to 1476
Columns: 140 entries, zero_crossing_rate_c1 to gamma_c14
dtypes: float64(140)
memory usage: 2.8 MB


## 4. 调参

In [44]:
default_params = xgb.get_params()
# 打印参数
print("Default Parameters:")
for param, value in default_params.items():
    print(f"{param}: {value}")

Default Parameters:
objective: multi:softprob
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
device: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: None
feature_types: None
gamma: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: None
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: None
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
multi_strategy: None
n_estimators: None
n_jobs: None
num_parallel_tree: None
random_state: None
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None


In [ ]:
from sklearn.model_selection import GridSearchCV

# 1. 确定learning_rate和n_estimators
param_test1 = {
 'learning_rate': [0.2, 0.25 ,0.300000012, 0.35],
 'n_estimators': [90, 95, 100, 105]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(), \
    param_grid = param_test1, scoring='accuracy', cv=3)
gsearch1.fit(pd.concat([X_train, X_test],axis=0), pd.concat([y_train,y_test],axis=0))
print(f"best params: {gsearch1.best_params_}\nbest score:  {gsearch1.best_score_}") 

In [ ]:
params = {
    'learning_rate': 0.35,
    'n_estimators': 90
}

model1 = XGBClassifier(**params)
model1.fit(X_train, y_train)

y_train_pred = model1.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = model1.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)

acc_train, acc_test

In [ ]:
from sklearn.model_selection import GridSearchCV

# 2. 确定max_depth和min_child_weight
params = {
    'learning_rate': 0.35,
    'n_estimators': 90
}

param_test2 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

gsearch2 = GridSearchCV(estimator = XGBClassifier(**param), \
    param_grid = param_test2, scoring='accuracy', cv=3)
gsearch2.fit(pd.concat([X_train, X_test],axis=0), pd.concat([y_train,y_test],axis=0))
print(f"best params: {gsearch2.best_params_}\nbest score:  {gsearch2.best_score_}") 

In [ ]:
params = {
    'learning_rate': 0.35,
    'n_estimators': 90,
    'max_depth': 9, 
    'min_child_weight': 5
}

model2 = XGBClassifier(**params)
model2.fit(X_train, y_train)

y_train_pred = model2.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = model2.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)

acc_train, acc_test

In [ ]:
from sklearn.model_selection import GridSearchCV

# 3. 确定gamma
params = {
    'learning_rate': 0.35,
    'n_estimators': 90,
    'max_depth': 9, 
    'min_child_weight': 5
}

param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gsearch3 = GridSearchCV(estimator = XGBClassifier(**param), \
    param_grid = param_test3, scoring='accuracy', cv=3)
gsearch3.fit(pd.concat([X_train, X_test],axis=0), pd.concat([y_train,y_test],axis=0))
print(f"best params: {gsearch3.best_params_}\nbest score:  {gsearch3.best_score_}") 

In [ ]:
from sklearn.model_selection import GridSearchCV

# 4. 确定reg_alpha
param = {
    'learning_rate': 0.3,
    'n_estimators': 100,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0.0
}

param_test4 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}

gsearch4 = GridSearchCV(estimator = XGBClassifier(**param), \
    param_grid = param_test4, scoring='accuracy', cv=3)
gsearch4.fit(pd.concat([X_train, X_test],axis=0), pd.concat([y_train,y_test],axis=0))
print(f"best params: {gsearch4.best_params_}\nbest score:  {gsearch4.best_score_}") 

In [ ]:
params = {
    'learning_rate': 0.3,
    'n_estimators': 100,
    'max_depth': 9, 
    'min_child_weight': 5,
    'reg_alpha': 0.01
}

model4 = XGBClassifier(**params)
model4.fit(X_train, y_train)

y_train_pred = model4.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = model4.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)

acc_train, acc_test

## 5. corss-session validation

In [3]:
import pandas as pd
from feature_engineering import *
from load_lyh_data import *

config_se2 = {
    'fs_raw' : 250,  # 原始数据采样频率
    'fs_down' : 125,  # 下采样频率 
    't_of_trial' : 4, # 一个trial的时间, 单位：s
    'start': 2, # trial有效点起始index
    'end': 2+1000, # trial有效点结尾index
    'channel_index_list' : np.arange(2, 2+14),
    'file_label_dict' : {
        "nothing_orginal_v3(500).npy": 3,
        "left_orginal_v3(500).npy": 0,
        "right_orginal_v3(500).npy": 1,
        "leg_orginal_v3(500).npy": 2,
        }, # 字典类型, 为data_dict的key到label的映射
    'n_Classes' : 4, #类别数目
    'pre_preparation_save_path': '/mnt/workspace/Baseline/lyh_data/session2/Standard_input', # 预处理结果保存路径
    'Bandpass_filter_params' : { 'lowcut' : 0.05, 'highcut' : 40, 'fs' : 125, 'order' : 2 }, # 带通滤波器参数
    'Preprocessed_save_path' : "/mnt/workspace/Baseline/lyh_data/session2/Preprocessed_data",
    'Feature_extracted_save_path' : '/mnt/workspace/Baseline/lyh_data/session2/Feature_extracted'
}

session2_path = os.getcwd() + "/lyh_data/session2/raw_data"
selected_file_list = ["nothing_orginal_v3(500).npy", "left_orginal_v3(500).npy", \
        "right_orginal_v3(500).npy", "leg_orginal_v3(500).npy"]
raw_data_dict_se2 = read_folder_npy_data(session2_path, selected_file_list)
# for key, value in raw_data_dict_se2.items():
#     print(key, raw_data_dict_se2[key].shape)
standard_input_data_list_se2 = pre_preparation(raw_data_dict_se2, config_se2)
preprocessed_data_se2 = pre_processing(standard_input_data_list_se2, config_se2)
dataset_np_se2 = np.array(preprocessed_data_se2)
dataset_np_se2 = dataset_np_se2.reshape(-1, 14, 500)
label_list_se2 = [0] * 500 + [1] * 500 + [2] * 500 + [3] * 500
data_df_se2 = pd.DataFrame({'raw data': list(dataset_np_se2), 'label': label_list_se2})
feature_extracting(data_df_se2, config_se2)


Loaded nothing_orginal_v3(500).npy: Shape=(500,), Dtype=object
Loaded left_orginal_v3(500).npy: Shape=(500,), Dtype=object
Loaded right_orginal_v3(500).npy: Shape=(500,), Dtype=object
Loaded leg_orginal_v3(500).npy: Shape=(500,), Dtype=object


/home/pai/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 500, using nperseg = 500
  warnings.warn('nperseg = {0:d} is greater than input length '
/mnt/workspace/Baseline/feature_engineering.py:359: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/mnt/workspace/Baseline/feature_engineering.py:359: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np

/mnt/workspace/Baseline/lyh_data/session2/Feature_extracted


In [120]:
data_df_se2.head(3)

,raw data,label,zero_crossing_rate_c1,mean_c1,std_c1,first_order_diff_c1,second_order_diff_c1,zero_crossing_rate_c2,mean_c2,std_c2,...,delta_c13,theta_c13,alpha_c13,beta_c13,gamma_c13,delta_c14,theta_c14,alpha_c14,beta_c14,gamma_c14
0,"[[-171.92995397467593, -181.74033097702963, -1...",0,0.042,107.731969,185.775271,7.594397,7.586411,0.010,-11.025577,91.180513,...,1639.018452,174.971362,136.149951,35.013434,14.632055,4870.985397,160.828484,103.684852,60.626268,20.026094
1,"[[-30.121299914698152, -24.106581923264933, -2...",0,0.006,-37.488891,135.927983,7.046242,7.229164,0.018,10.854331,61.111281,...,1665.550478,58.505241,43.228221,28.384698,12.173210,1133.777647,101.500896,49.663985,47.602825,16.372983
2,"[[-155.5794131919552, -174.5132547430685, -174...",0,0.002,-125.399781,444.205181,10.773724,8.910344,0.022,-121.558822,104.092615,...,3447.098512,312.548469,231.488804,74.438022,29.395010,5150.666712,178.931276,221.866543,80.819852,34.082534


In [4]:
# 构造数据集
from sklearn.model_selection import train_test_split

# 打乱数据集
shuffled_df_se2 = data_df_se2.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df_se2, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)
lgm.save_model("/mnt/workspace/Baseline/classifier/model_to_load/xgb_4s_se2.json")

# 手工特征提取+xgboost
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.9575)

In [5]:
# cross session
se1_path = '/mnt/workspace/Baseline/lyh_data/session1/Feature_extracted/feature_extracted.csv'
data_df_se1 = pd.read_csv(se1_path)

shuffled_df_se1 = data_df_se1.sample(frac=1)
shuffled_df_se2 = data_df_se2.sample(frac=1)

y_se1 = shuffled_df_se1['label']
X_se1 = shuffled_df_se1.drop(columns=['raw data', 'label'])
y_se2 = shuffled_df_se2['label']
X_se2 = shuffled_df_se2.drop(columns=['raw data', 'label'])

lgm1 = XGBClassifier()
lgm1.fit(X_se1, y_se1)
lgm1.save_model("/mnt/workspace/Baseline/classifier/model_to_load/xgb_4s_se1.json")

# session1 train, session2 test
y_train_pred = lgm1.predict(X_se1)
acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

y_test_pred = lgm1.predict(X_se2)
acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)
print("cross session vaildation(session1 train, session2 test):")
print(acc_train, acc_test)

lgm2 = XGBClassifier()
lgm2.fit(X_se2, y_se2)
lgm2.save_model("/mnt/workspace/Baseline/classifier/model_to_load/xgb_4s_se2.json")

# session1 train, session2 test
y_train_pred = lgm2.predict(X_se2)
acc_train = (y_se2 == y_train_pred).sum() / len(y_se2)

y_test_pred = lgm2.predict(X_se1)
acc_test = (y_se1 == y_test_pred).sum() / len(y_se1)
print("cross session vaildation(session2 train, session1 test):")
print(acc_train, acc_test)

cross session vaildation(session1 train, session2 test):
1.0 0.3625
cross session vaildation(session2 train, session1 test):
1.0 0.41333333333333333


In [6]:
train_df_se1, test_df_se1 = train_test_split(shuffled_df_se1, \
    test_size=0.2, random_state=42)
train_df_se2, test_df_se2 = train_test_split(shuffled_df_se2, \
    test_size=0.2, random_state=42)
train_df = pd.concat([train_df_se1, train_df_se2], axis=0)
test_df = pd.concat([test_df_se1, test_df_se2], axis=0)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)
lgm.save_model("/mnt/workspace/Baseline/classifier/model_to_load/xgb_4s_se1se2.json")

# mixed-session
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

<class 'pandas.core.frame.DataFrame'>
Index: 2560 entries, 870 to 1658
Columns: 142 entries, raw data to gamma_c14
dtypes: float64(140), int64(1), object(1)
memory usage: 2.8+ MB


(1.0, 0.94375)

In [24]:
# 构造数据集
from sklearn.model_selection import train_test_split

data_df_se1 = data_df_se1.drop(data_df_se1[data_df_se1['label'] == 3].index)
# 打乱数据集
shuffled_df_se1 = data_df_se1.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df_se1, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)

# 手工特征提取+xgboost
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.9833333333333333)

In [27]:
# 构造数据集
from sklearn.model_selection import train_test_split

data_df_se2 = data_df_se2.drop(data_df_se2[data_df_se2['label'] == 3].index)
# 打乱数据集
data_df_se2 = data_df_se2.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(data_df_se2, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)

# 手工特征提取+xgboost
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.98)

In [31]:
# cross session
data_df_se1 = data_df_se1.drop(data_df_se1[data_df_se1['label'] == 3].index)
data_df_se2 = data_df_se2.drop(data_df_se2[data_df_se2['label'] == 3].index)
shuffled_df_se1 = data_df_se1.sample(frac=1, random_state=42)
shuffled_df_se2 = data_df_se2.sample(frac=1, random_state=42)

y_se1 = shuffled_df_se1['label']
X_se1 = shuffled_df_se1.drop(columns=['raw data', 'label'])
y_se2 = shuffled_df_se2['label']
X_se2 = shuffled_df_se2.drop(columns=['raw data', 'label'])

lgm1 = XGBClassifier()
lgm1.fit(X_se1, y_se1)

# session1 train, session2 test
y_train_pred = lgm1.predict(X_se1)
acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

y_test_pred = lgm1.predict(X_se2)
acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)
print("cross session vaildation(session1 train, session2 test):")
print(acc_train, acc_test)

lgm2 = XGBClassifier()
lgm2.fit(X_se2, y_se2)

# session1 train, session2 test
y_train_pred = lgm2.predict(X_se2)
acc_train = (y_se2 == y_train_pred).sum() / len(y_se2)

y_test_pred = lgm2.predict(X_se1)
acc_test = (y_se1 == y_test_pred).sum() / len(y_se1)
print("cross session vaildation(session2 train, session1 test):")
print(acc_train, acc_test)

cross session vaildation(session1 train, session2 test):
1.0 0.464
cross session vaildation(session2 train, session1 test):
1.0 0.4255555555555556


In [32]:
data_df_se1 = data_df_se1.drop(data_df_se1[data_df_se1['label'] == 3].index)
data_df_se2 = data_df_se2.drop(data_df_se2[data_df_se2['label'] == 3].index)
shuffled_df_se1 = data_df_se1.sample(frac=1, random_state=42)
shuffled_df_se2 = data_df_se2.sample(frac=1, random_state=42)

train_df_se1, test_df_se1 = train_test_split(shuffled_df_se1, \
    test_size=0.2, random_state=42)
train_df_se2, test_df_se2 = train_test_split(shuffled_df_se2, \
    test_size=0.2, random_state=42)
train_df = pd.concat([train_df_se1, train_df_se2], axis=0)
test_df = pd.concat([test_df_se1, test_df_se2], axis=0)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)

# mixed-session
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.9708333333333333)

In [39]:
data_df_se1 = data_df_se1.drop(data_df_se1[data_df_se1['label'] == 3].index)
data_df_se2 = data_df_se2.drop(data_df_se2[data_df_se2['label'] == 3].index)
shuffled_df_se1 = data_df_se1.sample(frac=1, random_state=42)
shuffled_df_se2 = data_df_se2.sample(frac=1, random_state=42)

test_df_se2, train_df_se2 = train_test_split(shuffled_df_se2, \
    test_size=0.1, random_state=42)
train_df = pd.concat([data_df_se1, train_df_se2], axis=0)
test_df = test_df_se2
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)

# mixed-session
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.882962962962963)

In [41]:
y_train.shape, y_test.shape

((1050,), (1350,))

## 6. bcic数据集

In [123]:
import pandas as pd
from xgboost import XGBClassifier

subs1 = ['s001', 's002', 's003', 's004']
subs2 = ['e001', 'e002', 'e003', 'e004']

csv_file_floder = '/mnt/workspace/Baseline/bci42a_data/Feature_extracted'
acc_t, acc_e = [], []

for i in range(len(subs1)):
    train_set = pd.read_csv(csv_file_floder + '/' + subs1[i] + '.csv')
    test_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = train_set.sample(frac=1, random_state=42)
    test_set = test_set.sample(frac=1, random_state=42)

    y_se1 = train_set['label']
    X_se1 = train_set.drop(columns=['raw data', 'label'])
    y_se2 = test_set['label']
    X_se2 = test_set.drop(columns=['raw data', 'label'])

    xgb = XGBClassifier()
    xgb.fit(X_se1, y_se1)

    # session1 train, session2 test
    y_train_pred = xgb.predict(X_se1)
    acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

    y_test_pred = xgb.predict(X_se2)
    acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)

    acc_t.append(round(acc_train, 2))
    acc_e.append(round(acc_test, 2))

    print(f"Results in {subs1[i]}::: acc in train: {round(acc_train, 2)}, acc in test: {round(acc_test, 2)}")

print(f"Total resuts::: acc in train: {round(sum(acc_t) / len(acc_t), 2)}, acc in test: {round(sum(acc_e) / len(acc_e), 2)}")


Results in s001::: acc in train: 1.0, acc in test: 0.49
Results in s002::: acc in train: 1.0, acc in test: 0.22
Results in s003::: acc in train: 1.0, acc in test: 0.56
Results in s004::: acc in train: 1.0, acc in test: 0.4
Total resuts::: acc in train: 1.0, acc in test: 0.42


In [15]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

subs1 = ['s001', 's002', 's003', 's004']
subs2 = ['e001', 'e002', 'e003', 'e004']

csv_file_floder = '/mnt/workspace/Baseline/bci42a_data/Feature_extracted'
acc_t, acc_e = [], []

## mix-session
for i in range(len(subs1)):
    train_set = pd.read_csv(csv_file_floder + '/' + subs1[i] + '.csv')
    test_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = train_set.sample(frac=1, random_state=42)
    test_set = test_set.sample(frac=1, random_state=42)

    test_df_se2, train_df_se2 = train_test_split(test_set, \
    test_size=0.1, random_state=42)
    train_set = pd.concat([train_set, train_df_se2], axis=0)
    test_set = test_df_se2

    y_se1 = train_set['label']
    X_se1 = train_set.drop(columns=['raw data', 'label'])
    y_se2 = test_set['label']
    X_se2 = test_set.drop(columns=['raw data', 'label'])

    xgb = XGBClassifier()
    xgb.fit(X_se1, y_se1)

    # session1 train, session2 test
    y_train_pred = xgb.predict(X_se1)
    acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

    y_test_pred = xgb.predict(X_se2)
    acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)

    acc_t.append(round(acc_train, 2))
    acc_e.append(round(acc_test, 2))

    print(f"Results in {subs1[i]}::: acc in train: {round(acc_train, 2)}, acc in test: {round(acc_test, 2)}")

print(f"Total resuts::: acc in train: {round(sum(acc_t) / len(acc_t), 2)}, acc in test: {round(sum(acc_e) / len(acc_e), 2)}")


Results in s001::: acc in train: 1.0, acc in test: 0.5
Results in s002::: acc in train: 1.0, acc in test: 0.2
Results in s003::: acc in train: 1.0, acc in test: 0.54
Results in s004::: acc in train: 1.0, acc in test: 0.4
Total resuts::: acc in train: 1.0, acc in test: 0.41


In [124]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

subs1 = ['s001', 's002', 's003', 's004']
subs2 = ['e001', 'e002', 'e003', 'e004']

csv_file_floder = '/mnt/workspace/Baseline/bci42a_data/Feature_extracted'
acc_t, acc_e = [], []

## mix-session
for i in range(len(subs1)):
    train_set = pd.read_csv(csv_file_floder + '/' + subs1[i] + '.csv')
    test_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = train_set.sample(frac=1, random_state=42)
    test_set = test_set.sample(frac=1, random_state=42)

    train_df_se2, test_df_se2 = train_test_split(test_set, \
    test_size=0.2, random_state=42)
    train_df_se1, test_df_se1 = train_test_split(train_set, \
    test_size=0.2, random_state=42)
    train_set = pd.concat([train_df_se1, train_df_se2], axis=0)
    test_set = pd.concat([test_df_se1, test_df_se2], axis=0)

    y_se1 = train_set['label']
    X_se1 = train_set.drop(columns=['raw data', 'label'])
    y_se2 = test_set['label']
    X_se2 = test_set.drop(columns=['raw data', 'label'])

    xgb = XGBClassifier()
    xgb.fit(X_se1, y_se1)

    # session1 train, session2 test
    y_train_pred = xgb.predict(X_se1)
    acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

    y_test_pred = xgb.predict(X_se2)
    acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)

    acc_t.append(round(acc_train, 2))
    acc_e.append(round(acc_test, 2))

    print(f"Results in {subs1[i]}::: acc in train: {round(acc_train, 2)}, acc in test: {round(acc_test, 2)}")

print(f"Total resuts::: acc in train: {round(sum(acc_t) / len(acc_t), 2)}, acc in test: {round(sum(acc_e) / len(acc_e), 2)}")


Results in s001::: acc in train: 1.0, acc in test: 0.57
Results in s002::: acc in train: 1.0, acc in test: 0.28
Results in s003::: acc in train: 1.0, acc in test: 0.41
Results in s004::: acc in train: 1.0, acc in test: 0.44
Total resuts::: acc in train: 1.0, acc in test: 0.42


In [19]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

subs1 = ['s001', 's002', 's003', 's004']
subs2 = ['e001', 'e002', 'e003', 'e004']

csv_file_floder = '/mnt/workspace/Baseline/bci42a_data/Feature_extracted'
acc_t, acc_e = [], []

print("******In-session Validation (Session 1)******")

## in-session
for i in range(len(subs1)):
    train_set = pd.read_csv(csv_file_floder + '/' + subs1[i] + '.csv')
    # test_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = train_set.sample(frac=1, random_state=42)

    train_set, test_set = train_test_split(train_set, \
    test_size=0.2, random_state=42)

    y_se1 = train_set['label']
    X_se1 = train_set.drop(columns=['raw data', 'label'])
    y_se2 = test_set['label']
    X_se2 = test_set.drop(columns=['raw data', 'label'])

    xgb = XGBClassifier()
    xgb.fit(X_se1, y_se1)

    # session1 train, session2 test
    y_train_pred = xgb.predict(X_se1)
    acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

    y_test_pred = xgb.predict(X_se2)
    acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)

    acc_t.append(round(acc_train, 2))
    acc_e.append(round(acc_test, 2))

    print(f"Results in {subs1[i]}::: acc in train: {round(acc_train, 2)}, acc in test: {round(acc_test, 2)}")

print(f"Total resuts::: acc in train: {round(sum(acc_t) / len(acc_t), 2)}, acc in test: {round(sum(acc_e) / len(acc_e), 2)}")


******In-session Validation (Session 1)******
Results in s001::: acc in train: 1.0, acc in test: 0.43
Results in s002::: acc in train: 1.0, acc in test: 0.36
Results in s003::: acc in train: 1.0, acc in test: 0.5
Results in s004::: acc in train: 1.0, acc in test: 0.26
Total resuts::: acc in train: 1.0, acc in test: 0.39


In [30]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

subs1 = ['s001', 's002', 's003', 's004']
subs2 = ['e001', 'e002', 'e003', 'e004']

csv_file_floder = '/mnt/workspace/Baseline/bci42a_data/Feature_extracted'
acc_t, acc_e = [], []

print("******In-session Validation (Session 2)******")

## in-session
for i in range(len(subs1)):
    # train_set = pd.read_csv(csv_file_floder + '/' + subs1[i] + '.csv')
    # test_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = pd.read_csv(csv_file_floder + '/' + subs2[i] + '.csv')
    train_set = train_set.sample(frac=1, random_state=42)

    train_set, test_set = train_test_split(train_set, \
    test_size=0.2, random_state=42)

    y_se1 = train_set['label']
    X_se1 = train_set.drop(columns=['raw data', 'label'])
    y_se2 = test_set['label']
    X_se2 = test_set.drop(columns=['raw data', 'label'])

    xgb = XGBClassifier()
    xgb.fit(X_se1, y_se1)

    # session1 train, session2 test
    y_train_pred = xgb.predict(X_se1)
    acc_train = (y_se1 == y_train_pred).sum() / len(y_se1)

    y_test_pred = xgb.predict(X_se2)
    acc_test = (y_se2 == y_test_pred).sum() / len(y_se2)

    acc_t.append(round(acc_train, 2))
    acc_e.append(round(acc_test, 2))

    print(f"Results in {subs1[i]}::: acc in train: {round(acc_train, 2)}, acc in test: {round(acc_test, 2)}")

print(f"Total resuts::: acc in train: {round(sum(acc_t) / len(acc_t), 2)}, acc in test: {round(sum(acc_e) / len(acc_e), 2)}")


******In-session Validation (Session 2)******
Results in s001::: acc in train: 1.0, acc in test: 0.5
Results in s002::: acc in train: 1.0, acc in test: 0.26
Results in s003::: acc in train: 1.0, acc in test: 0.48
Results in s004::: acc in train: 1.0, acc in test: 0.47
Total resuts::: acc in train: 1.0, acc in test: 0.43


# 1-s数据

In [2]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch, lfilter
from copy import deepcopy
from entropy import *
import entropy
from scipy.signal import welch
from scipy.io import loadmat, savemat


class EEG_DataLoader:
    '''对raw data进行数据预处理, 包括下采样、选择通道、滤波、时域特征提取(过零率、均值、标准差、一阶\二阶差分)、
    频域特征提取(传统五频道法)
    X_raw: raw data, 类型: List[二维numpy arrray, shape=(channels, time_points)]
    y : X_raw 对应的标签
    config: 配置信息
    data_df: 预处理之后的数据, 其中两个colums=['raw data', 'label']不用来训练, 其他的colums都是提取出来的特征,
    用来训练.
    API: self.feature_engineering(): 返回一个datafrmae类型的数据, 可用来训练。
    '''
    def __init__(self, X_raw, y, config):
        self.X_raw = X_raw
        self.y = y
        self.config = config
        self.standard_input_data = []
        self.pre_processed_data = []
        self.data_df = None

    def pre_preparation(self):
        '''
        预处理: 下采样+选择通道
        '''

        def down_sample(data, start, end, fs_raw, fs_down):
            '''
            下采样
            
            param---
            data: numpy数组, 一个trial, shape = (n_channels, n_times)
            start: trial有效点起始index
            end: trial有效点结尾index
            
            return---
            data: 下采样之后的数组
            '''
            
            step = fs_raw // fs_down
            data = data[:, start: end: step]
            
            return data

        def selected_channel(data, channel_index_list):
            '''
            选择有效的通道
            
            param---
            data: numpy数组, 一个trial, shape = (n_channels, n_times)
            channel_index_list: list类型, 有效通道的index
            '''
            return data[channel_index_list, :]

        for trial in self.X_raw:
            trial = down_sample(np.array(trial), self.config['start'], self.config['end'], self.config['fs_raw'], config['fs_down'])
            trial = selected_channel(trial, self.config['channel_index_list'])
            self.standard_input_data.append(trial)

        if self.config['pre_preparation_save_path'] != '':
            if not os.path.exists(self.config['pre_preparation_save_path']):
                os.makedirs(self.config['pre_preparation_save_path'])
            np.save(self.config['pre_preparation_save_path'] + "/standard_input_data" + ".npy", self.standard_input_data)

    def pre_processing(self, butter_order = 2):
        
        def butter_bandpass_filter(data, config):
            fa = 0.5 * config['fs']
            low = config['lowcut'] / fa
            high = config['highcut'] / fa
            b, a = butter(config['order'], [low, high], btype='band')
            ret = []
            for line in data:
                ret.append(filtfilt(b, a, line))
            return np.array(ret)

        def iirnotch_filter(data, fs = 125, Q = 30, f_cut = 50.0):
            ret = []
            b, a = iirnotch(f_cut, Q, fs)
            for line in data:
                ret.append(lfilter(b,a, line))
            return np.array(ret)

        
        for i in range(len(self.standard_input_data)):
            assert self.standard_input_data[i].shape == (14, 125)
            x = butter_bandpass_filter(self.standard_input_data[i], self.config['Bandpass_filter_params'])
            x = iirnotch_filter(x)
            self.pre_processed_data.append(x)

        if self.config['Preprocessed_save_path'] != '':
            if not os.path.exists(self.config['Preprocessed_save_path']):
                os.makedirs(self.config['Preprocessed_save_path'])
            np.save(self.config['Preprocessed_save_path'] + "/pre_processed_data" + ".npy", self.pre_processed_data)

    def time_domain_feature(self):
        '''
        提取时域特征
        '''
        # 计算过零率
        def zero_crossing_rate(trials):
            '''
            计算一个trials(二维列表)各个通道的过零率, 返回一个list
            '''

            def compute(signal):
                '''
                计算一维信号的过零率
                '''
                crossings = np.where(np.diff(np.sign(signal)))[0]
                zero_crossing_rate = len(crossings) / len(signal)
                return zero_crossing_rate

            ret = [compute(trials[i]) for i in range(len(trials))]
            return np.array(ret)
        
        # 计算均值

        def calculate_channel_means(eeg_data):
            """
            计算每个通道的均值。

            参数：
            - eeg_data: 二维 NumPy 数组，表示 EEG 数据，形状为 (n_channels, n_points)。

            返回：
            - channel_means: 一维 NumPy 数组，包含每个通道的均值。
            """
            # 计算每个通道的均值，axis=2 表示在样本点上求均值
            channel_means = np.mean(eeg_data, axis=1)

            return channel_means

        # 计算标准差
        def calculate_channel_stds(eeg_data):
            """
            计算每个通道的均值。

            参数：
            - eeg_data: 二维 NumPy 数组，表示 EEG 数据，形状为 (n_channels, n_points)。

            返回：
            - channel_means: 一维 NumPy 数组，包含每个通道的均值。
            """
            # 计算每个通道的均值，axis=2 表示在样本点上求均值
            channel_stds = np.std(eeg_data, axis=1)

            return channel_stds
        
        def calculate_first_order_diff(eeg_data):

            first_order_diff = np.sum(np.abs(np.diff(eeg_data, axis=1)), axis=1) / (eeg_data.shape[1] - 1)

            return first_order_diff

        def calculate_second_order_diff(eeg_data):
            
            second_order_diff = np.sum(np.abs(np.diff(eeg_data, n=2, axis=1)), axis=1) / (eeg_data.shape[1] - 2)

            return second_order_diff
        
        zero_crossing_list = []
        means_list = []
        stds_list = []
        first_order_diff_list = []
        second_order_diff_list = []

        for _, row in self.data_df.iterrows():
            zero_crossing = zero_crossing_rate(row['raw data'])
            zero_crossing_list.append(zero_crossing)
            means = calculate_channel_means(row['raw data'])
            means_list.append(means)
            stds = calculate_channel_stds(row['raw data'])
            stds_list.append(stds)
            first_diff = calculate_first_order_diff(row['raw data'])
            first_order_diff_list.append(first_diff)
            second_diff = calculate_second_order_diff(row['raw data'])
            second_order_diff_list.append(second_diff)
        
        nChannels = zero_crossing_list[0].shape[0]
        for i in range(nChannels):
            self.data_df[f'zero_crossing_rate_c{i+1}'] = np.array(zero_crossing_list)[:,i]
            self.data_df[f'mean_c{i+1}'] = np.array(means_list)[:,i]
            self.data_df[f'std_c{i+1}'] = np.array(stds_list)[:,i]
            self.data_df[f'first_order_diff_c{i+1}'] = np.array(first_order_diff_list)[:,i]
            self.data_df[f'second_order_diff_c{i+1}'] = np.array(second_order_diff_list)[:,i]

    def freq_domain_feature(self):
        '''
        提取时域特征

        参数:
        - eeg_dataset: Dataframe类型
        '''

        def five_band_energy(eeg_data, fs=125):
    
            energy = []
            
            for eeg_signal in eeg_data:
                # 计算功率谱密度（PSD）
                frequencies, psd = welch(eeg_signal, fs, nperseg=1024)

                # 定义频带边界
                delta_band = (0.5, 4)
                theta_band = (4, 8)
                alpha_band = (8, 14)
                beta_band = (14, 30)
                gamma_band = (30, 60)

                # 计算每个频带内的能量
                energy.append(np.trapz(psd[(frequencies >= delta_band[0]) & (frequencies <= delta_band[1])], \
                    frequencies[(frequencies >= delta_band[0]) & (frequencies <= delta_band[1])]))
                energy.append(np.trapz(psd[(frequencies >= theta_band[0]) & (frequencies <= theta_band[1])], \
                    frequencies[(frequencies >= theta_band[0]) & (frequencies <= theta_band[1])]))
                energy.append(np.trapz(psd[(frequencies >= alpha_band[0]) & (frequencies <= alpha_band[1])], \
                    frequencies[(frequencies >= alpha_band[0]) & (frequencies <= alpha_band[1])]))
                energy.append(np.trapz(psd[(frequencies >= beta_band[0]) & (frequencies <= beta_band[1])], \
                    frequencies[(frequencies >= beta_band[0]) & (frequencies <= beta_band[1])]))
                energy.append(np.trapz(psd[(frequencies >= gamma_band[0]) & (frequencies <= gamma_band[1])], \
                    frequencies[(frequencies >= gamma_band[0]) & (frequencies <= gamma_band[1])]))

            return np.array(energy)
        
        energy = []

        for _, row in self.data_df.iterrows():
            
            energy.append(five_band_energy(row['raw data']))

        freq_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
        nFreqs = len(freq_names)
        nChannels = energy[0].shape[0] // nFreqs
        
        for i in range(nChannels):
            for j in range(nFreqs):
                self.data_df[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]

    def feature_extracting(self):
        self.data_df = pd.DataFrame({'raw data': self.pre_processed_data, 'label': self.y})
        self.time_domain_feature()
        self.freq_domain_feature()
        if self.config['Feature_extracted_save_path'] != '':
            if not os.path.exists(self.config['Feature_extracted_save_path']):
                os.makedirs(self.config['Feature_extracted_save_path'])
            self.data_df.to_csv(self.config['Feature_extracted_save_path'] + "/feature_extracted_data.csv")

    def feature_engineering(self):
        self.pre_preparation()
        self.pre_processing()
        self.feature_extracting()
        return self.data_df



In [3]:
import pandas as pd
import numpy as np

config = {
        'fs_raw' : 250,  # 原始数据采样频率
        'fs_down' : 125,  # 下采样频率 
        't_of_trial' : 1, # 一个trial的时间, 单位：s
        'start': 2, # trial有效点起始index
        'end': 2+250, # trial有效点结尾index
        'channel_index_list' : np.arange(2, 2+14),
        'pre_preparation_save_path': '/mnt/workspace/Standard_input', # 预处理结果保存路径
        'Bandpass_filter_params' : { 'lowcut' : 0.05, 'highcut' : 40, 'fs' : 125, 'order' : 2 }, # 带通滤波器参数
        'Preprocessed_save_path' : "/mnt/workspace/Preprocessed_data",
        'Feature_extracted_save_path' : '/mnt/workspace/Feature_extracted'
    }
data_path_list = [
                   "/mnt/workspace/Baseline/lyh_data/session2/raw_data/left_orginal_v3(500).npy",\
                    "/mnt/workspace/Baseline/lyh_data/session2/raw_data/right_orginal_v3(500).npy",\
                    "/mnt/workspace/Baseline/lyh_data/session2/raw_data/leg_orginal_v3(500).npy" ]
X = []
for path in data_path_list:
    d = np.load(path, allow_pickle=True)
    for trial in d:
        X.append(np.array(trial).T)
y = [0] * 500 + [1] * 500 + [2] * 500
data_loader = EEG_DataLoader(X, y, config)
data_df = data_loader.feature_engineering()

/home/pai/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 125, using nperseg = 125
  warnings.warn('nperseg = {0:d} is greater than input length '
/tmp/ipykernel_408/1093053317.py:245: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data_df[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/tmp/ipykernel_408/1093053317.py:245: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data_df[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+

In [4]:
# 构造数据集
from sklearn.model_selection import train_test_split

# 打乱数据集
shuffled_df = data_df.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])

from xgboost import XGBClassifier

lgm = XGBClassifier()
lgm.fit(X_train, y_train)
lgm.save_model("/mnt/workspace/Baseline/classifier/model_to_load/xgb_1s_se2.json")

# 手工特征提取+xgboost
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

(1.0, 0.9166666666666666)

In [3]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [9]:
data_set_path = '/mnt/workspace/Baseline/classifier/data_set_to_load/feature_extracted_se2_1s.csv'
data_df = pd.read_csv(data_set_path)

data_df = data_df.sample(frac=1)
train_df, test_df = train_test_split(data_df, test_size=0.85)
y_train = train_df['label']
X_train = train_df.drop(columns=['Unnamed: 0', 'raw data', 'label'])
y_test = test_df['label']
X_test = test_df.drop(columns=['Unnamed: 0', 'raw data', 'label'])

In [13]:
clf = SVC()
clf.fit(X_train, y_train)

acc_train = accuracy_score(y_train, clf.predict(X_train))
acc_test = accuracy_score(y_test, clf.predict(X_test))
cm = confusion_matrix(y_pred=clf.predict(X_test), y_true=y_test)

print("******RESULT******")
print("Model: SVM")
print(f"Acc: train->{acc_train} test->{acc_test}")
print(f"Confusion Matrix: {cm}")

******RESULT******
Model: SVM
Acc: train->0.3408333333333333 test->0.33666666666666667
Confusion Matrix: [[  3   0  95]
 [  0   0 104]
 [  0   0  98]]


In [10]:
clf = XGBClassifier()
clf.fit(X_train, y_train)

acc_train = accuracy_score(y_train, clf.predict(X_train))
acc_test = accuracy_score(y_test, clf.predict(X_test))
cm = confusion_matrix(y_pred=clf.predict(X_test), y_true=y_test)

print("******RESULT******")
print(f"Model: XGBClassifier, nTrials for training: {y_train.shape[0]}, nTrials for testing: {y_test.shape[0]}")
print(f"Acc: train->{acc_train} test->{acc_test}")
print(f"Confusion Matrix: {cm}")

******RESULT******
Model: XGBClassifier, nTrials for training: 225, nTrials for testing: 1275
Acc: train->1.0 test->0.8650980392156863
Confusion Matrix: [[368  37  30]
 [ 42 350  36]
 [  7  20 385]]
